In [ ]:
import cv2
import time
import os

def video_to_frames(input_loc, output_loc):
    try:
        os.mkdir(output_loc)
    except OSError:
        pass
    # Log the time
    time_start = time.time()
    # Start capturing the feed
    cap = cv2.VideoCapture(input_loc)
    # Find the number of frames
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    print ("Number of frames: ", video_length)
    count = 0
    print ("Converting video..\n")
    # Start converting the video
    while cap.isOpened():
        # Extract the frame
        ret, frame = cap.read()
        # Write the results back to output location.
        cv2.imwrite(output_loc + "/%#05d.jpg" % (count+1), frame)
        count = count + 1
        # If there are no more frames left
        if (count > (video_length-1)):
            # Log the time again
            time_end = time.time()
            # Release the feed
            cap.release()
            # Print stats
            print ("Done extracting frames.\n%d frames extracted" % count)
            print ("It took %d seconds forconversion." % (time_end-time_start))
            break

In [ ]:
video_to_frames(vid_dir + 'train.mp4', './commaai/train')

In [ ]:
vid_dir = './commaai/'
test_path = './commaai/test/'
train_path = './commaai/data/train/'
valid_path = './commaai/data/valid/'
data_dir = './commaai/data/'

with open(vid_dir + 'train.txt', 'r', encoding='ISO-8859-1') as fh:
    lister = fh.readlines()

In [ ]:
#make sure we take the data in sequence
def make_val_set(PATH, p):
    PATH = PATH if PATH[-1] == '/' else PATH+'/'
    list_of_files = sorted(os.listdir(f"{vid_dir}train/"))
    n_idxs = len(list_of_files) * (1 - p)
    selected_files = list_of_files[int(n_idxs):]
    for file in selected_files:
        try:
            os.rename(f"{PATH}train/{file}", f"{PATH}valid/{file}")
        except FileNotFoundError as e:
            pass

In [ ]:
make_val_set(data_dir, 0.1)

In [ ]:
idxs = [x for x in range(18000, 20398, 1)] # grab ~10% of images for val

In [ ]:
from fastai.vision.all import *

In [ ]:
def get_speed(img_name):
    img_name = str(img_name)
    img_only = img_name.replace('commaai/data/train/', '')
    idx = int(img_only.replace('.jpg',''))
    return float(lister[idx].replace('\n', ''))

In [ ]:
speeder = DataBlock(
    blocks=(ImageBlock, RegressionBlock),
    get_items=get_image_files,
    get_y=get_speed,
    splitter=IndexSplitter(idxs),
    batch_tfms=[*aug_transforms(size=(240,320)), 
                Normalize.from_stats(*imagenet_stats)]
)

In [ ]:
dls = speeder.dataloaders(train_path)

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=rmse)
learn.loss = MSELossFlat()

In [ ]:
learn.fine_tune(12)